# Prologue - Question 1

The goal is to explain why candidate X is ranked higher than candidate Y. The ranking is based on a weighted sum of scores. Instead of showing the raw formula, we want to provide a (1-1) trade-off explanation.

Definition:
- Pros ($P$): Subjects where $x$ scores better than $y$ (Contribution $> 0$).
- Cons ($C$): Subjects where $x$ scores worse than $y$ (Contribution $< 0$).
- Trade-off (1-1): A pair $(p, c)$ where $p \in Pros$ and $c \in Cons$ such that the advantage in $p$ is strictly greater than the disadvantage in $c$ (i.e., $contribution(p) + contribution(c) > 0$).
- Explanation: A set of disjoint trade-offs that covers every element in the set of Cons.

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import gurobipy as gp

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

## Prologue data: Xavier (x) vs Yvonne (y)

Courses: A..G  
Weights and scores are exactly those shown in the prologue table.


In [2]:
courses = ["A","B","C","D","E","F","G"]

course_names = {
    "A": "Anatomie",
    "B": "Biologie",
    "C": "Chirurgie",
    "D": "Diagnostic",
    "E": "Epidemiologie",
    "F": "Forensic Pathology",
    "G": "Génétique",
}

w = {"A": 8, "B": 7, "C": 7, "D": 6, "E": 6, "F": 5, "G": 6}

x = {"A": 85, "B": 81, "C": 71, "D": 69, "E": 75, "F": 81, "G": 88}  # Xavier
y = {"A": 81, "B": 81, "C": 75, "D": 63, "E": 67, "F": 88, "G": 95}  # Yvonne

df_scores = pd.DataFrame(
    {
        "course": courses,
        "name": [course_names[c] for c in courses],
        "weight": [w[c] for c in courses],
        "x": [x[c] for c in courses],
        "y": [y[c] for c in courses],
    }
)
df_scores


,course,name,weight,x,y
0,A,Anatomie,8,85,81
1,B,Biologie,7,81,81
2,C,Chirurgie,7,71,75
3,D,Diagnostic,6,69,63
4,E,Epidemiologie,6,75,67
5,F,Forensic Pathology,5,81,88
6,G,Génétique,6,88,95


## Step 1 — Compute contributions


In [3]:
df = df_scores.copy()
df["delta"] = df["weight"] * (df["x"] - df["y"])
df["sign"] = np.where(df["delta"] > 0, "pro", np.where(df["delta"] < 0, "con", "neutral"))
df[["course","name","weight","x","y","delta","sign"]]


,course,name,weight,x,y,delta,sign
0,A,Anatomie,8,85,81,32,pro
1,B,Biologie,7,81,81,0,neutral
2,C,Chirurgie,7,71,75,-28,con
3,D,Diagnostic,6,69,63,36,pro
4,E,Epidemiologie,6,75,67,48,pro
5,F,Forensic Pathology,5,81,88,-35,con
6,G,Génétique,6,88,95,-42,con


## Step 2 — Build feasible (1–1) trade-offs

A feasible edge $(p,c)$ exists iff:
- $p \in pros(x,y)$, $c \in cons(x,y)$
- $\Delta_p + \Delta_c > 0$

We will build the edge list $A$.


In [4]:
pros = df.loc[df["delta"] > 0, "course"].tolist()
cons = df.loc[df["delta"] < 0, "course"].tolist()
neutral = df.loc[df["delta"] == 0, "course"].tolist()

delta = dict(zip(df["course"], df["delta"]))

A = []
for p in pros:
    for c in cons:
        margin = delta[p] + delta[c]
        if margin > 0:
            A.append((p, c, margin))

print("pros:", pros)
print("cons:", cons)
print("neutral:", neutral)
print("\nNumber of feasible (1-1) trade-offs:", len(A))

df_A = (
    pd.DataFrame(A, columns=["P (pro)", "C (con)", "margin"])
      .sort_values("margin", ascending=False)
)
df_A


pros: ['A', 'D', 'E']
cons: ['C', 'F', 'G']
neutral: ['B']

Number of feasible (1-1) trade-offs: 6


,P (pro),C (con),margin
3,E,C,20
4,E,F,13
1,D,C,8
5,E,G,6
0,A,C,4
2,D,F,1


## Step 3 — Linear optimization model (LP, integral by structure)

Decision variables: $z_{p,c} \in [0,1]$ for each feasible pair $(p,c) \in A$.

Constraints:
1. Each $c \in cons$ is covered exactly once:
$$
\sum_{p} z_{p,c} = 1
$$
2. Each $p \in pros$ is used at most once:
$$
\sum_{c} z_{p,c} \le 1
$$

Objective: maximize total margin $\sum (\Delta_p+\Delta_c) z_{p,c}$.


In [5]:
from gurobipy import GRB

def solve_explanation_1_1_gurobi(delta, pros, cons, feasible_edges, verbose=False):
    # feasible_edges: list of (p, c, margin)
    edges = [(p, c) for (p, c, m) in feasible_edges]
    margins = np.array([m for (_, _, m) in feasible_edges], dtype=float)

    n = len(edges)
    if n == 0:
        return {"status": "infeasible", "message": "No feasible trade-off edges.", "edges": edges}

    # Build model
    m = gp.Model("explanation_1_1")
    m.Params.OutputFlag = 1 if verbose else 0

    # Decision variables
    z = m.addVars(n, vtype=GRB.BINARY, name="z")

    # Objective: maximize total margin
    m.setObjective(gp.quicksum(margins[j] * z[j] for j in range(n)), GRB.MAXIMIZE)

    # Each con must be covered exactly once
    con_to_js = {con: [] for con in cons}
    for j, (p, c) in enumerate(edges):
        if c in con_to_js:
            con_to_js[c].append(j)

    for con in cons:
        js = con_to_js.get(con, [])
        # If a con has no feasible incident edges, infeasible immediately
        if len(js) == 0:
            return {
                "status": "infeasible",
                "message": f"No feasible trade-off covers con={con}.",
                "edges": edges,
                "margins": margins,
            }
        m.addConstr(gp.quicksum(z[j] for j in js) == 1, name=f"cover_con[{con}]")

    # Each pro can be used at most once
    pro_to_js = {pro: [] for pro in pros}
    for j, (p, c) in enumerate(edges):
        if p in pro_to_js:
            pro_to_js[p].append(j)

    for pro in pros:
        js = pro_to_js.get(pro, [])
        if len(js) > 0:
            m.addConstr(gp.quicksum(z[j] for j in js) <= 1, name=f"use_pro[{pro}]")

    # Optimize
    m.optimize()

    out = {"edges": edges, "margins": margins, "model_status": int(m.Status)}

    if m.Status == GRB.OPTIMAL:
        z_val = np.array([z[j].X for j in range(n)], dtype=float)
        chosen = [(edges[j][0], edges[j][1], float(margins[j]), float(z_val[j]))
                  for j in range(n) if z_val[j] > 0.5]
        out.update({"status": "feasible", "chosen": chosen, "z": z_val, "obj": float(m.ObjVal)})
        return out

    if m.Status == GRB.INFEASIBLE:
        out.update({"status": "infeasible"})
        return out

    out.update({"status": "infeasible"})
    return out
    
sol = solve_explanation_1_1_gurobi(delta, pros, cons, A, verbose=False)
sol["status"], sol.get("message",""), sol.get("obj", None)

Restricted license - for non-production use only - expires 2027-11-29


('feasible', '', 11.0)

## Step 4 — Extract the (1–1) explanation and verify the definition


In [6]:
def verify_explanation(chosen_pairs, pros, cons):
    P_used = [p for (p, c, m, z) in chosen_pairs]
    C_used = [c for (p, c, m, z) in chosen_pairs]

    ok_cover = (sorted(C_used) == sorted(cons))
    ok_disjoint_pro = (len(P_used) == len(set(P_used)))
    ok_disjoint_con = (len(C_used) == len(set(C_used)))

    return {
        "covers_all_cons": ok_cover,
        "disjoint_pros": ok_disjoint_pro,
        "disjoint_cons": ok_disjoint_con,
    }

if sol["status"] == "feasible":
    chosen = sol["chosen"]
    df_chosen = pd.DataFrame(
        [{"P": p, "C": c, "margin": m, "ΔP": delta[p], "ΔC": delta[c]} for (p,c,m,_) in chosen]
    ).sort_values("margin", ascending=False)
    display(df_chosen)

    checks = verify_explanation(chosen, pros, cons)
    print("Checks:", checks)

    explanation = [(p,c) for (p,c,_,_) in chosen]
    print("\nExplanation =", explanation)


,P,C,margin,ΔP,ΔC
2,E,G,6.0,48,-42
0,A,C,4.0,32,-28
1,D,F,1.0,36,-35


Checks: {'covers_all_cons': True, 'disjoint_pros': True, 'disjoint_cons': True}

Explanation = [('A', 'C'), ('D', 'F'), ('E', 'G')]


## Step 5 — Certificate of non-existence (Hall-violation subset)

When no explanation (1–1) exists, the feasible edges $A$ define a bipartite graph between:
- Left = cons(x,y)
- Right = pros(x,y)

A (1–1) explanation exists **iff** there is a matching that covers **all cons**.

If not, we return a **Hall certificate**: a subset $S \subseteq cons$ with
$$
|N(S)| < |S|
$$
where $N(S)$ is the neighborhood of $S$ in the feasible-edge graph.


In [8]:
def hall_certificate_from_feasible_edges(cons, pros, feasible_edges):
    G2 = nx.Graph()
    G2.add_nodes_from(cons, bipartite=0)
    G2.add_nodes_from(pros, bipartite=1)
    G2.add_edges_from([(c, p) for (p,c,_) in feasible_edges])

    matching = nx.algorithms.bipartite.matching.maximum_matching(G2, top_nodes=cons)
    matched_cons = [c for c in cons if c in matching]

    if len(matched_cons) == len(cons):
        return {"status": "feasible", "matching": matching, "certificate": None}

    matched_edges = {(c, matching[c]) for c in cons if c in matching}

    from collections import deque
    unmatched = [c for c in cons if c not in matching]
    Z_left = set(unmatched)
    q = deque(unmatched)
    Z_right = set()

    while q:
        c = q.popleft()
        for p in G2.neighbors(c):
            if (c, p) in matched_edges:
                continue
            if p in Z_right:
                continue
            Z_right.add(p)
            if p in matching:
                c2 = matching[p]
                if c2 not in Z_left:
                    Z_left.add(c2)
                    q.append(c2)

    S = Z_left
    N = set()
    for c in S:
        N |= set(G2.neighbors(c))

    return {
        "status": "infeasible",
        "matching_size": len(matched_cons),
        "cons_size": len(cons),
        "hall_S": sorted(S),
        "hall_neighbors": sorted(N),
        "hall_sizes": {"|S|": len(S), "|N(S)|": len(N)},
        "matching": matching,
    }

cert = hall_certificate_from_feasible_edges(cons, pros, A)
cert


{'status': 'feasible',
 'matching': {'C': 'A', 'G': 'E', 'F': 'D', 'A': 'C', 'D': 'F', 'E': 'G'},
 'certificate': None}